In [ ]:
# https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da
# needed to run this python3 -m spacy download en_core_web_sm

#%pylab inline

import datetime
import pandas as pd
import pickle as pk
from scipy import sparse as sp
import sqlite3

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

# Attempt Charting
import matplotlib.pyplot as plt
import matplotlib.axes as ax

#import seaborn as sns

from dateutil import parser

In [ ]:
conn = sqlite3.connect('temp.sqlite3')
c = conn.cursor()
c.execute("SELECT * FROM documents")
totalArticles = str(len(c.fetchall()))
print(totalArticles + " Total Articles")

In [ ]:
p_df = pd.read_sql_query("SELECT * FROM documents", conn)
conn.close()

In [ ]:
#print(p_df.head())

l = len(pd.array(p_df['text']))
take = l

docs = pd.array(p_df['text'])[:take]
pubdates = pd.array(p_df['publishdate'])[:take]

final = {}
words = set()

for x,date in zip(docs,pubdates):
    #print(x)
    article = nlp(x)
    items = [x.text for x in article.ents]
    
    #print([(X, X.ent_iob_, X.ent_type_) for X in article])
    #print([(X.text, X.label_) for X in article.ents])
    
    #print(Counter(items).most_common(20))
    y=dict(Counter(items).most_common(20))
    words.update(y.keys())
    date = parser.parse(date)
    date = str(date.year) + "-" + str(date.month) + "-" + str(date.day)
    if (date in final):
        final[date]={**final[date],**y}
    else:
        final[date]=y
     
words = list(words)

#print(final)
#print(words)

In [ ]:
dates_str = [key for key in final]
dates_str.sort()

build = {}

for word in words:
    build[word]=[]

for date in dates_str:
    entry = final[date]
    for item in words:
        if item in entry:
            value = entry[item]
            build[item].append(value)
        else:
            build[item].append(0)
            
print(build)

In [ ]:
plt.ylabel("Word Frequency")
plt.xlabel("Date")

x_values = [datetime.datetime.strptime(date,"%Y-%m-%d").date() for date in dates_str]

plt.xticks(rotation=90)
for item in build:
    plt.plot(x_values,build[item])
    
plt.legend(list(build.keys()),bbox_to_anchor=(1, 1))

plt.show()